In [78]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import IsolationForest

In [79]:
# Load the datasets
X = np.load("Xtrain_Regression_Part2.npy")
X_TESTE = np.load("Xtest_Regression_Part2.npy")
Y = np.load("Ytrain_Regression_Part2.npy")

CÓDIGO PARA ENTREGAR

In [80]:
model = linear_model.LinearRegression()

score = 10

X_for = X
Y_for = Y
removed = 0

# train and predict
model.fit(X_for, Y_for)

Y_pred = model.predict(X_for)

diff = abs(Y_for - Y_pred.reshape(Y_for.shape[0], 1))**2

while removed < 9:
    index = np.where(diff == max(diff))

    Y_for = np.delete(Y_for, index, 0)
    X_for = np.delete(X_for, index, 0)

    # train and predict
    model.fit(X_for, Y_for)

    Y_pred = model.predict(X_for)

    diff = abs(Y_for - Y_pred.reshape(Y_for.shape[0], 1))**2

    removed += 1

model.fit(X_for, Y_for)
Y_pred = model.predict(X_for)

current_score = -1 * np.average(
    cross_val_score(model, X_for, Y_for, scoring="neg_mean_squared_error", cv=5)
)
score = min(score, current_score)

print(f"best is {score}")


best is 0.009545727005401334
